In [13]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OrdinalEncoder

### 参考torch.optim.lr_scheduler.ReduceLROnPlateau

In [14]:
X = datasets.fetch_covtype().data[:5000]
y = datasets.fetch_covtype().target[:5000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(X_train.shape, y_train.shape)

enc = OrdinalEncoder()
y_train = enc.fit_transform(y_train.reshape(-1, 1)).reshape(-1, )
y_test = enc.transform(y_test.reshape(-1, 1)).reshape(-1, )
print(np.unique(y_train))  # 7分类任务

X_train, X_test, y_train, y_test = tf.constant(X_train), tf.constant(X_test), tf.constant(y_train), tf.constant(y_test)

(4500, 54) (4500,)
[0. 1. 2. 3. 4. 5. 6.]


In [15]:
inputs = tf.keras.Input(shape=(54,))
x = tf.keras.layers.Dense(256, activation='tanh')(inputs)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128)(x)
predictions = tf.keras.layers.Dense(7)(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 54)]              0         
                                                                 
 dense_8 (Dense)             (None, 256)               14080     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               32896     
                                                                 
 dense_10 (Dense)            (None, 128)               16512     
                                                                 
 dense_11 (Dense)            (None, 7)                 903       
                                                                 
Total params: 64,391
Trainable params: 64,391
Non-trainable

In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [17]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  # 'loss'/'val_loss'或accuracy/val_accuracy(根据metrics)
                                         mode='min',  # 由monitory='val_loss',故设置mode='min'
                                         # factor by which the learning rate will be reduced. new_lr = lr * factor.
                                         factor=0.5,
                                         # number of epochs with no improvement after which learning rate will be reduced.
                                         patience=3,
                                         # threshold for measuring the new optimum, to only focus on significant changes.
                                         min_delta=0.0001,
                                         # number of epochs to wait before resuming normal operation after lr has been reduced.
                                         cooldown=0,
                                         min_lr=0,  # 学习率的下限
                                         # int. 0: quiet, 1: update messages.
                                         verbose=1)

]
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test),
          # List of keras.callbacks.Callback instances.
          callbacks=callbacks)

Epoch 1/50
141/141 [==============================] - 1s 4ms/step - loss: 1.3782 - accuracy: 0.4422 - val_loss: 1.2004 - val_accuracy: 0.5280 - lr: 0.0010
Epoch 2/50
141/141 [==============================] - 0s 3ms/step - loss: 1.2441 - accuracy: 0.4949 - val_loss: 1.1712 - val_accuracy: 0.5320 - lr: 0.0010

In [18]:
# 学习率衰减情况
model.history.history['lr']

[0.001,
 0.001,
 0.001,
 0.001,
 0.001,
 0.001,
 0.001,
 0.001,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.0005,
 0.00025,
 0.00025,
 0.00025,
 0.00025,
 0.00025,
 0.00025,
 0.00025,
 0.00025,
 0.00025,
 0.00025,
 0.00025,
 0.00025]